## Data Wrangling & Preprocessing (Python & Enhanced SQL)

This notebook focuses on cleaning, transforming, and preparing the raw customer churn data for subsequent exploratory data analysis (EDA) and machine learning modeling. We'll leverage both SQL for initial transformations (simulating real-world database interactions) and Python's Pandas library for more complex data manipulation and feature engineering.

### Refine Data Extraction with SQL (Initial Cleaning & Transformation)

In a real-world scenario, data often resides in databases. We'll simulate that by connecting to our previously created SQLite database (`telco_churn.db`) and using SQL to perform some initial cleaning and transformations before loading the data into a Pandas DataFrame. This includes handling the `TotalCharges` column, converting `Churn` and `SeniorCitizen` to binary (0/1), and transforming `gender` to `gender_Male`.

In [3]:
import pandas as pd
import sqlite3
import os

# Ensure have a connection to your SQLite database

conn = sqlite3.connect('Telco-Customer-Churn.db')

sql_query = """
SELECT
    "customerID",
    CASE WHEN "gender" = 'Male' THEN 1 ELSE 0 END AS gender_Male,
    CASE WHEN "SeniorCitizen" = 'Yes' THEN 1 ELSE 0 END AS SeniorCitizen,
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    -- Handle TotalCharges: replace empty strings with '0' and cast to REAL
    CAST(REPLACE("TotalCharges", ' ', '0') AS REAL) AS TotalCharges,
    -- Convert Churn to binary 1/0
    CASE WHEN "Churn" = 'Yes' THEN 1 ELSE 0 END AS Churn
FROM telco_churn;
"""

# Load data from the SQL query result into a Pandas DataFrame
df = pd.read_sql(sql_query, conn)

# Close the database connection (important to release the file lock)
conn.close()

print("DataFrame loaded from SQL with initial cleaning:")
print(df.head())
print(df.info())

DataFrame loaded from SQL with initial cleaning:
   customerID  gender_Male  SeniorCitizen Partner Dependents  tenure  \
0  7590-VHVEG            0              0     Yes         No       1   
1  5575-GNVDE            1              0      No         No      34   
2  3668-QPYBK            1              0      No         No       2   
3  7795-CFOCW            1              0      No         No      45   
4  9237-HQITU            0              0      No         No       2   

  PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0           No  No phone service             DSL             No  ...   
1          Yes                No             DSL            Yes  ...   
2          Yes                No             DSL            Yes  ...   
3           No  No phone service             DSL            Yes  ...   
4          Yes                No     Fiber optic             No  ...   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0      

### Further Data Wrangling and Preprocessing with Pandas

After initial SQL processing, Pandas is crucial for more complex transformations, ensuring data consistency, and additional feature engineering that might be harder or less efficient directly in SQL.

In [5]:
# Drop 'customerID' as it's an identifier and not a feature for the model
if 'customerID' in df.columns:
    df = df.drop('customerID', axis=1)
    print("\n'customerID' column dropped.")

# Handle any remaining NaNs in 'TotalCharges' (e.g., if there were original NULLs or other non-convertible issues)
# Although handled in SQL, this is a robust step in Pandas.
# Using direct assignment to avoid the 'A value is trying to be set on a copy...' FutureWarning.
df['TotalCharges'] = df['TotalCharges'].fillna(0)
print("\nNaNs in 'TotalCharges' handled (filled with 0).")

# Convert other binary categorical columns (like 'Partner', 'Dependents' etc.) to 0/1 if they were not handled in SQL
# This handles cases where SQL might not have converted them or if new data comes in.
# Check your df.info() and df.head() to confirm their current state. If a column is already 0/1, this will do nothing.
binary_cols_to_map = [
    'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies',
    'PaperlessBilling'
]

for col in binary_cols_to_map:
    # Only map if the column still contains 'Yes'/'No' or similar object types
    if col in df.columns and df[col].dtype == 'object' and df[col].isin(['Yes', 'No', 'No internet service', 'No phone service']).any():
        df[col] = df[col].map({'Yes': 1, 'No': 0, 'No internet service': 0, 'No phone service': 0})
        print(f"'{col}' converted to 0/1.")

print("\nDataFrame after binary column mapping and final NaN check:")
print(df.head())
print(df.info())


'customerID' column dropped.

NaNs in 'TotalCharges' handled (filled with 0).
'Partner' converted to 0/1.
'Dependents' converted to 0/1.
'PhoneService' converted to 0/1.
'MultipleLines' converted to 0/1.
'OnlineSecurity' converted to 0/1.
'OnlineBackup' converted to 0/1.
'DeviceProtection' converted to 0/1.
'TechSupport' converted to 0/1.
'StreamingTV' converted to 0/1.
'StreamingMovies' converted to 0/1.
'PaperlessBilling' converted to 0/1.

DataFrame after binary column mapping and final NaN check:
   gender_Male  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0            0              0        1           0       1             0   
1            1              0        0           0      34             1   
2            1              0        0           0       2             1   
3            1              0        0           0      45             0   
4            0              0        0           0       2             1   

   MultipleLines InternetService  On

### Feature Engineering

Create new features that might provide more predictive power or make patterns clearer for the model.

In [7]:
# Feature Engineering: Create 'TenureGroup'
# Group 'tenure' (number of months customer has been with the company) into meaningful categories.
bins = [0, 12, 24, 36, 48, 60, 72] # Define bins for tenure groups
labels = ['0-12 Mths', '13-24 Mths', '25-36 Mths', '37-48 Mths', '49-60 Mths', '61-72 Mths'] # Labels for the bins
# The 'right=False' argument means the interval includes the left bound but excludes the right bound
df['TenureGroup'] = pd.cut(df['tenure'], bins=bins, labels=labels, right=False, include_lowest=True)
print("\n'TenureGroup' feature engineered:")
print(df[['tenure', 'TenureGroup']].head())
print(df['TenureGroup'].value_counts())

# Verify that 'TenureGroup' is a 'category' dtype, which will be handled by get_dummies
print(f"\nTenureGroup dtype: {df['TenureGroup'].dtype}")


'TenureGroup' feature engineered:
   tenure TenureGroup
0       1   0-12 Mths
1      34  25-36 Mths
2       2   0-12 Mths
3      45  37-48 Mths
4       2   0-12 Mths
TenureGroup
0-12 Mths     2069
61-72 Mths    1121
13-24 Mths    1047
25-36 Mths     876
49-60 Mths     820
37-48 Mths     748
Name: count, dtype: int64

TenureGroup dtype: category


### Save Intermediate DataFrame (Pre-Encoding)

Before performing One-Hot Encoding, we'll save the current state of the DataFrame. This version contains the original categorical features (e.g., 'Contract', 'InternetService', 'PaymentMethod', 'TenureGroup') alongside numerical features and the 'Churn' target. This is particularly useful for intuitive visualizations of categorical variables during Exploratory Data Analysis (EDA) in the next step (`3.0_EDA_and_Feature_Selection.ipynb`).

In [9]:
# Define your absolute output directory for processed data
output_dir = 'D:/My Data/Family Storages/Rudra/Education/Projects/Project 2 telco custimer churn/Data/'

# Ensure the directory exists. `exist_ok=True` prevents an error if the directory already exists.
os.makedirs(output_dir, exist_ok=True)

# Define the full path for the pre-encoding file
file_path_pre_encoding = os.path.join(output_dir, 'telco_churn_pre_encoding.csv')

# Save the DataFrame with original categorical features for EDA
df.to_csv(file_path_pre_encoding, index=False)
print(f"\nDataFrame before one-hot encoding saved to '{file_path_pre_encoding}'")


DataFrame before one-hot encoding saved to 'D:/My Data/Family Storages/Rudra/Education/Projects/Project 2 telco custimer churn/Data/telco_churn_pre_encoding.csv'


### One-Hot Encoding for Machine Learning Models

Machine learning models typically require numerical input. We will convert all remaining `object` (string) or `category` type columns into numerical format using One-Hot Encoding. This creates new binary columns for each category, which is suitable for model training.

In [11]:
# Identify categorical columns that still need one-hot encoding
# These are typically 'object' or 'category' dtype columns after previous steps.
# 'gender_Male' and 'SeniorCitizen' are already numerical (0/1).
categorical_cols_to_encode = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nColumns to One-Hot Encode: {categorical_cols_to_encode}")

# Perform One-Hot Encoding
# drop_first=True helps to avoid multicollinearity (one category can be inferred from the others)
df_encoded = pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True)

print("\nDataFrame after One-Hot Encoding (df_encoded):")
print(df_encoded.head())
print(df_encoded.info())


Columns to One-Hot Encode: ['InternetService', 'Contract', 'PaymentMethod', 'TenureGroup']

DataFrame after One-Hot Encoding (df_encoded):
   gender_Male  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0            0              0        1           0       1             0   
1            1              0        0           0      34             1   
2            1              0        0           0       2             1   
3            1              0        0           0      45             0   
4            0              0        0           0       2             1   

   MultipleLines  OnlineSecurity  OnlineBackup  DeviceProtection  ...  \
0              0               0             1                 0  ...   
1              0               1             0                 1  ...   
2              0               1             1                 0  ...   
3              0               1             0                 1  ...   
4              0               0      

### Final Save of the Encoded DataFrame

This `df_encoded` is your final, clean, and numerically prepared DataFrame. It will be used for Exploratory Data Analysis (EDA) where numerical relationships and correlations are examined, and directly for Machine Learning model building and evaluation (`4.0_Machine_Learning_Model_Building_and_Evaluation.ipynb`).

In [13]:
# Define the full path for the final processed file
file_path_processed = os.path.join(output_dir, 'telco_churn_processed.csv')

# Save the final processed DataFrame
df_encoded.to_csv(file_path_processed, index=False)
print(f"\nFinal processed DataFrame saved to '{file_path_processed}'")


Final processed DataFrame saved to 'D:/My Data/Family Storages/Rudra/Education/Projects/Project 2 telco custimer churn/Data/telco_churn_processed.csv'


##### Outcomes:
###### Dataset is now clean, free of missing values (or they are appropriately handled), and all features are in a numerical format.
###### Successfully demonstrated proficiency in both SQL for initial data preparation and Pandas for more complex wrangling and feature engineering.
###### `df_encoded` is your final, ready-to-use DataFrame for the next steps in Exploratory Data Analysis and Machine Learning.